# Dimension reduction using PCA

# imprting all necessary libararies 

In [2]:
import pandas as pd
import numpy as np
import os, sys
sys.path.append('../')
from transformers import AutoModel
from sentence_transformers import SentenceTransformer
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from src.other_function import functions
from src.model_evaluation import evaluation
from src.dimensionality_reduction_class import dimensionality_reduction


/Users/msaqib/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# object of other_function class

In [4]:
func_object=functions()

# Creating the object of evaluation class

In [ ]:
eval_object=evaluation()

# Loading the data

In [23]:
df=pd.read_csv("..//Data/processed_data.csv")


list_data = df['tokenized_docstring'].tolist()

# Loading the testing data

In [24]:
queries=pd.read_csv("..//Data/query.csv")

# Model name

In [8]:
model='all-mpnet-base-v2'

# Creating the model with less dimension 

In [9]:
num_dimension=128

dim_reduce_object=dimensionality_reduction(model,num_dimension)

dim_reduce_object.reduce_dimension()

# Generating the embeddings of less dimension model

In [11]:
model_new_dim_128=SentenceTransformer('models/new_128_dim_model')
func_object.get_model_embedding(model_new_dim_128,"new_128_dim_model")

starting_time : 1713367290.36
ending_time : 1713368045.71
Embedding_shape : (113884, 128)
total_time_taken for embedding generation:  755.35 seconds 


# size of embeddings


In [6]:
emb_size=func_object.get_embedding_size('..//embeddings/embeddings_new_128_dim_model.npy')
print(f"size of embeddings : {emb_size} ")

size of embeddings : 55.61 MB 


# Response on the testing data

In [31]:
# order in which column of databse will be present
column_order = ['Query', 'Docstrings', 'Code','Match']

#creating the dataframe for dim_128_model which consists of for particular query all top code and
#match column shows whether it is correctly retrieved or not
dim_128_query_response=pd.DataFrame(columns=column_order)

model_dim_128=SentenceTransformer('models/new_128_dim_model')

emb_128_dim=np.load("embeddings/embeddings_new_128_dim_model.npy")

print(emb_128_dim.shape)

for i in range(len(queries)):
    #getting the particular query from queries 
    query=queries.loc[i].Questions
    
    # getting the table dataframe that consists of docstring and code 
    #that has top match with query using cosine similarity
    table=eval_object.get_top_code_and_docstring(query,emb_128_dim,model_dim_128,list_data,df)
    
    #adding the column query with all values as 
    table=add_column(table,query)
    
    #adding columns Match that defines whether the code for given query is correctly retrieved or not
    table=add_column(table,'Match')
    
    #reordering the table for better view
    table=reorder_columns(table)
    
    #iterating in the table
    for index,row in table.iterrows():
        
        #fetching the current code for query
        result = row['Code']
        
        #getting the response from claude api
        response=check_response(query,row['Code'])
    
        #Checking the value present in the response generated by the claude
        if 'YES' in response.content:
            response='YES'
        else :
            response='NO'
            
        #Giving the value of response in match column for current row
        table.at[index,'Match']=response
    
    #merging the table dataframe and dim_128_query_response in single dataframe
    dim_128_query_response= pd.concat([dim_128_query_response, table], ignore_index=True)

(113884, 128)


# MAP of model with less dimension

In [32]:
total_yes = (dim_128_query_response['Match'] == 'YES').sum()
total_no = (dim_128_query_response['Match'] == 'NO').sum()

accuracy_dim_128=(total_yes/(total_yes+total_no))*100

print(f"MAP@10 (mean average precision) of 6 layer model : {accuracy_dim_128}")


MAP@10 (mean average precision) of 6 layer model : 51.40350877192983
